This notebook takes TX state lobbying records and tries to create a chart that shows what lobbyists favor which
political parties. The output is a gephi file called lobby.gexf. I started with these files: 

LaTran.csv, which is from the TEC here: https://www.ethics.state.tx.us/dfs/search_LOBBY.html.

tx_legislators.csv, from http://openstates.org/csv_downloads/

then made these changes with csvkit:

csvcut -c 1,4,5,6,15,14,13,16,17,11,31 LaTran.csv > TXTran.csv
csvcut -c 1,2,3,4,5,12,10,11,8,13 tx_legislators.csv > TXLeg.csv

In [1]:
import csv

file = 'data/TXTran.csv'
file2 = 'data/TXLeg.csv'

newFile = open(file)
newReader = csv.reader(newFile)
newData = list(newReader)

travel = []
leg = []

for row in newData[1:]:
    travel.append(row)

legFile = open(file2)
legReader = csv.reader(legFile)
legData = list(legReader)

for row in legData[1:]:
    leg.append(row)

In [2]:
from fuzzywuzzy import fuzz

travelFound = []

for item in travel:
    itemname = item[5] + ' ' + item[6]
    found = False
    for person in leg:
        personname = person[2] + ' ' + person[4]
        if itemname == personname or itemname == person[1]:
            found = True
            item.extend([person[0],person[5],person[6],person[7],person[8],person[9],personname])
            travelFound.append(item)
    if found == False:
        for person in leg:
            if fuzz.token_sort_ratio(itemname, personname) > 80:
                found = True
                item.extend([person[0],person[5],person[6],person[7],person[8],person[9],personname])
                print(itemname + ' fuzzy match ' + personname)
                travelFound.append(item)
    # if found == False:
        # print('No match found for ' + itemname)

print(travelFound[5])

['301', '2005', '00010205', 'Johnson, Thomas L.', 'Mr.', 'John', 'Whitmire', '', 'AGC of Texas 2005 Management Conference', 'TXL000211', 'Democratic', 'upper', '15', 'True', 'c2c2d651cb9f4bd3b00364f8d9b84715', 'John Whitmire']


In [3]:
import networkx as nx
G=nx.MultiDiGraph()

for row in travelFound:
    G.add_node(row[2], name=row[3], role="Lobbyist")
    G.add_node(row[9], name=row[-1], role=row[10])
    G.add_edge(row[2], row[9], gift="Travel", year=row[1], detail=row[8])


In [4]:
G.nodes(data=True)[:15]

[('TXL000463', {'name': 'Drew Springer', 'role': 'Republican'}),
 ('00041073', {'name': 'Howe, Billy', 'role': 'Lobbyist'}),
 ('TXL000209', {'name': 'Jeff Wentworth', 'role': ''}),
 ('TXL000323', {'name': 'Solomon Ortiz', 'role': ''}),
 ('TXL000352', {'name': 'Senfronia Thompson', 'role': 'Democratic'}),
 ('TXL000197', {'name': 'Mike Jackson', 'role': ''}),
 ('TXL000182', {'name': 'Kip Averitt', 'role': ''}),
 ('00066090', {'name': 'Anti-Defamation League', 'role': 'Lobbyist'}),
 ('00011961', {'name': 'Looney, Robert L.', 'role': 'Lobbyist'}),
 ('TXL000243', {'name': 'John Davis', 'role': ''}),
 ('TXL000227', {'name': 'Lon Burnam', 'role': ''}),
 ('00014717', {'name': 'Stewart, Michael K.', 'role': 'Lobbyist'}),
 ('TXL000253', {'name': 'Gary Elkins', 'role': 'Republican'}),
 ('00052912', {'name': 'Barton, Yvonne', 'role': 'Lobbyist'}),
 ('TXL000252', {'name': 'Rob Eissler', 'role': ''})]

In [5]:
G.is_directed()

True

In [6]:
G.node['TXL000211']

{'name': 'John Whitmire', 'role': 'Democratic'}

In [7]:
nx.degree(G)

{'00010038': 1,
 '00010066': 1,
 '00010205': 145,
 '00010273': 6,
 '00010798': 2,
 '00011961': 1,
 '00012695': 2,
 '00012985': 105,
 '00013335': 1,
 '00013547': 2,
 '00013628': 6,
 '00013729': 1,
 '00013743': 1,
 '00013745': 1,
 '00013799': 1,
 '00013802': 2,
 '00014432': 1,
 '00014450': 1,
 '00014668': 2,
 '00014717': 3,
 '00014943': 4,
 '00014950': 1,
 '00019849': 4,
 '00020272': 4,
 '00021009': 7,
 '00022360': 2,
 '00026158': 2,
 '00028590': 1,
 '00029965': 2,
 '00030016': 10,
 '00031603': 1,
 '00034750': 5,
 '00035019': 19,
 '00035073': 6,
 '00037179': 1,
 '00037658': 1,
 '00038496': 1,
 '00038847': 1,
 '00039050': 4,
 '00039150': 1,
 '00039254': 3,
 '00039361': 2,
 '00039432': 1,
 '00041073': 2,
 '00041247': 1,
 '00050783': 5,
 '00051186': 2,
 '00051202': 5,
 '00051707': 1,
 '00052056': 1,
 '00052535': 1,
 '00052771': 6,
 '00052912': 1,
 '00053477': 5,
 '00053551': 3,
 '00053669': 1,
 '00053964': 4,
 '00053966': 14,
 '00055096': 4,
 '00055793': 6,
 '00056285': 1,
 '00056338': 2,
 

In [8]:
H = nx.Graph(G)
answer = nx.connected_components(H)
for i in answer:
    print(i)

{'TXL000463', '00041073', 'TXL000209', 'TXL000243', 'TXL000197', 'TXL000182', '00014943', 'TXL000227', '00014717', '00013802', 'TXL000253', 'TXL000252', 'TXL000481', '00013799', '00010066', 'TXL000337', '00014668', 'TXL000383', 'TXL000282', '00039432', '00013745', 'TXL000424', '00037179', 'TXL000295', '00060783', 'TXL000345', '00041247', 'TXL000211', 'TXL000214', 'TXL000410', 'TXL000279', 'TXL000436', 'TXL000245', '00035073', '00068906', '00014950', 'TXL000199', 'TXL000242', '00021009', 'TXL000191', 'TXL000299', '00031603', '00014450', '00051202', '00039361', '00053551', 'TXL000361', '00052056', 'TXL000304', 'TXL000430', '00010205', 'TXL000298', 'TXL000296', '00067052', 'TXL000433', '00038847', 'TXL000241', '00052771', '00053964', '00061299', 'TXL000249', '00012695', 'TXL000280', '00055096', 'TXL000271', 'TXL000316', '00053477', '00030016', 'TXL000320', '00050783', 'TXL000210', 'TXL000470', '00010798', '00067923', 'TXL000194', 'TXL000212', 'TXL000389', 'TXL000324', 'TXL000379', 'TXL000

In [9]:
# This is the output. The lines after this are just experiments.

nx.readwrite.write_gexf(G,"lobby.gexf")

In [ ]:
# I tried making this CSV to output for graphing. But this isn't that useful because it puts all the 
# data on the edges, none on the nodes, so I didn't end up using it for anything.

outputFile = open('StateLobbyistTravel.csv', 'w', newline='')
outputWriter = csv.writer(outputFile)
outputWriter.writerow(['Record', 'Year', 'Source', 'LobbyName', 'Target','LegName','Party'])
for row in travelFound:
    outputWriter.writerow([row[0],row[1],row[2],row[3],row[9],row[-1],row[10]])
outputFile.close()

In [10]:
# Testing to see how many records there are where the legislator's party is unknown because OpenStates doesn't provide
# that info after they leave office.

noParty = []

for record in travel:
    if len(record) > 9:
        if record[10] == '':
            noParty.append(record)
        
len(noParty)

215

In [ ]:
# Here's where I tried using Neo4Jj

from py2neo import Graph

graph = Graph()
graph.delete_all()

In [ ]:
# This works to add nodes into a Neo4j database, but not relationships. Don't know why.

from py2neo import Node, Relationship

for record in travelFound:
    itemname = record[5] + ' ' + record[6]
    a = graph.merge(Node("Lobbyist", id=record[2], name=record[3]))
    b = graph.merge(Node("Legislator", id=record[9], name=itemname, party=record[10]))
    # Two attempted solutions below. neither of them is working.
    ab = graph.create(Path(a, "Travel", b))
    # graph.create(Relationship(a, b))